In [0]:
# output mount point generation
configs = {
  "fs.azure.account.key.adlsstoragehockey.blob.core.windows.net": "clFz+IqyE3a2urx/PnxFLcfk8vwUclZTGvY9GcGjVzI+t2/d6RSpUjGqaHIhEUyahCPGtmF+d4zL+AStz2XpCQ=="
}
dbutils.fs.mount(
  source = "wasbs://output@adlsstoragehockey.blob.core.windows.net/",
  mount_point = "/mnt/mount2",
  extra_configs=configs
)

True

In [0]:
df_src1 = spark.read.option("header", True).csv("/mnt/mount1/Hockey_Src1.csv")
df_src2 = spark.read.option("header", True).csv("/mnt/mount1/Hockey_Src2.csv")
df_date = spark.read.option("header", True).csv("/mnt/mount1/DIM.Date.Table.csv")

df_src1.show(3)
df_src2.show(3)
df_date.show(3)

+----+--------+-------------+-----------+---------+------------+--------+---------+---------+-----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+-----------------------+-------------+------------------+---------+
|Year|   Month|        Match|Player Name|Club Name|country Name|Position|Jersey_No| D. O. B.|Nationality|appearances|goals scored|goals assist|total shots|shots on target|fouls made|fouls suffered|yellow card|red card|goals saved|goals conceded(stopped)|total penalty|successful penalty|   salary|
+----+--------+-------------+-----------+---------+------------+--------+---------+---------+-----------+-----------+------------+------------+-----------+---------------+----------+--------------+-----------+--------+-----------+-----------------------+-------------+------------------+---------+
|2010| January|International|     Adrian|       NA| Netherlands|      LW|       88|30-Mar-85|Netherlands| 

In [0]:
from pyspark.sql.functions import row_number, col
from pyspark.sql.window import Window

In [0]:
df_src1.printSchema()


root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Match: string (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Club Name: string (nullable = true)
 |-- country Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey_No: string (nullable = true)
 |-- D. O. B.: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- appearances: string (nullable = true)
 |-- goals scored: string (nullable = true)
 |-- goals assist: string (nullable = true)
 |-- total shots: string (nullable = true)
 |-- shots on target: string (nullable = true)
 |-- fouls made: string (nullable = true)
 |-- fouls suffered: string (nullable = true)
 |-- yellow card: string (nullable = true)
 |-- red card: string (nullable = true)
 |-- goals saved: string (nullable = true)
 |-- goals conceded(stopped): string (nullable = true)
 |-- total penalty: string (nullable = true)
 |-- successful penalty: string (nullable = true)
 |-- sal

In [0]:
df_src2.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Match_Name: string (nullable = true)
 |-- League_Name: string (nullable = true)
 |-- Club_Name: string (nullable = true)
 |-- Coach: string (nullable = true)
 |-- Country Name: string (nullable = true)
 |-- Manager: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- appearances: string (nullable = true)
 |-- wins: string (nullable = true)
 |-- losts: string (nullable = true)
 |-- drawn: string (nullable = true)
 |-- clean sheets: string (nullable = true)
 |-- Net Worth: string (nullable = true)



In [0]:
df_date.printSchema()

root
 |-- date_key: string (nullable = true)
 |-- full_date: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_num_in_month: string (nullable = true)
 |-- day_num_overall: string (nullable = true)
 |-- day_name: string (nullable = true)
 |-- day_abbrev: string (nullable = true)
 |-- weekday_flag: string (nullable = true)
 |-- week_num_in_year: string (nullable = true)
 |-- week_num_overall: string (nullable = true)
 |-- week_begin_date: string (nullable = true)
 |-- week_begin_date_key: string (nullable = true)
 |-- fb_month: string (nullable = true)
 |-- month_num_overall: string (nullable = true)
 |-- month_name: string (nullable = true)
 |-- month_abbrev: string (nullable = true)
 |-- quarter: string (nullable = true)
 |-- fb_year: string (nullable = true)
 |-- yearmo: string (nullable = true)
 |-- fiscal_month: string (nullable = true)
 |-- fiscal_quarter: string (nullable = true)
 |-- fiscal_year: string (nullable = true)
 |-- last_day_in_month_flag: str

In [0]:
# League Dimention
df_league = df_src2.select("League_Name").distinct()

window_spec = Window.orderBy("League_Name")
dim_league = df_league.withColumn("League_ID", row_number().over(window_spec))

dim_league.write.option("header", True).mode("overwrite").csv("/mnt/mount2/dim_league")

In [0]:
# Club Dimension
# Unique Club + League + Manager + Owner
df_club = df_src2.select("Club_Name", "League_Name", "Manager", "Owner").distinct()

# Join League to get League_ID
dim_club = df_club.join(dim_league, on="League_Name", how="left")

# Add Club_ID
window_spec = Window.orderBy("Club_Name")
dim_club = dim_club.withColumn("Club_ID", row_number().over(window_spec))

dim_club.write.option("header", True).mode("overwrite").csv("/mnt/mount2/dim_club")

In [0]:
# Country Dimension
# Combine from both sources and Coach
df_country_1 = df_src1.select(col("country Name").alias("Country_Name"))
df_country_2 = df_src2.select(col("Country Name").alias("Country_Name"), "Coach")
df_country = df_country_1.union(df_country_2.select("Country_Name")).distinct()

# Join with coach info
df_country = df_country.join(df_country_2, on="Country_Name", how="left").dropDuplicates(["Country_Name"])

# Add ID
window_spec = Window.orderBy("Country_Name")
dim_country = df_country.withColumn("Country_ID", row_number().over(window_spec))

dim_country.write.option("header", True).mode("overwrite").csv("/mnt/mount2/dim_country")

In [0]:
# Match Type Dimension
df_match = df_src2.select("Match_Name").distinct()

window_spec = Window.orderBy("Match_Name")
dim_match_type = df_match.withColumn("Match_Type_ID", row_number().over(window_spec))

dim_match_type.write.option("header", True).mode("overwrite").csv("/mnt/mount2/dim_match_type")

In [0]:
# Player Dimension
# Select required columns from df_src1
player_df = df_src1.select(
    "Player Name", col("Club Name").alias("Club_Name"), col("country Name").alias("Country_Name"),
    "Position", col("Jersey_No").alias("Jersey_Number"),
    col("`D. O. B.`").alias("DOB"),    # FIXED LINE
    "Nationality"
).distinct()

# Join with dim_club and dim_country to get IDs
dim_player = player_df.join(dim_club.select("Club_ID", "Club_Name"), on="Club_Name", how="left") \
                      .join(dim_country.select("Country_ID", "Country_Name"), on="Country_Name", how="left")

# Add Player_ID
from pyspark.sql.window import Window
window_spec = Window.orderBy("Player Name")
dim_player = dim_player.withColumn("Player_ID", row_number().over(window_spec))

# Write to ADLS
dim_player.write.option("header", True).mode("overwrite").csv("/mnt/mount2/dim_player")

In [0]:
# Time Dimension
# Select only required columns
dim_time = df_date.select(
    col("date_key").alias("Date_Key"),
    col("fb_year").alias("Year"),
    col("quarter").alias("Quarter"),
    col("fb_month").alias("Month")
).distinct()

# Add Time_ID
window_spec = Window.orderBy("Date_Key")
dim_time = dim_time.withColumn("Time_ID", row_number().over(window_spec))

dim_time.write.option("header", True).mode("overwrite").csv("/mnt/mount2/dim_time")